In [1]:
from dotenv import load_dotenv
from openai import OpenAI
import os

load_dotenv()

True

In [2]:
client = OpenAI()

In [13]:
def create_assistant(instruction):
    assistant = client.beta.assistants.create(
        name='Chatbot Hoteleiro',
        instructions=instruction,
        tools=[{'type':'file_search'}],
        model='gpt-3.5-turbo'
    )
    return assistant

In [3]:
base_prompt_instruction = """Você é um atendente experiente de um hotel. \
    Responda perguntas variadas sobre hotéis, acomodações, pontos turíscos e outras \
    informações relevantes aos possíveis hóspedes.    
"""

In [26]:
# assistant = create_assistant(base_prompt_instruction)

In [4]:
# vector_store = client.beta.vector_stores.create(name='Paginas HTML de Hoteis')

In [5]:
# file_paths = [PATH_HTML]
# file_streams = [open(path, 'rb') for path in file_paths]

# file_batch = client.beta.vector_stores.file_batches.upload_and_poll(
#     vector_store_id=vector_store.id, files=file_streams
# )

# print(file_batch.status)

In [6]:
VECTOR_STORE_ID = 'vs_UVad5hh6Y8gCsQ0OTH57nCuw'
ASSISTANT_ID = 'asst_TXgxoREIAnrXfZKSxf8bkJoR'

assistant = client.beta.assistants.update(
    assistant_id=ASSISTANT_ID,
    tool_resources={'file_search':{'vector_store_ids':[VECTOR_STORE_ID]}}
)

In [7]:
thread = client.beta.threads.create(
    messages=[
        {
            'role':'user',
            'content':'Que pousada é essa? O que a pousada tem de bom? Quais os quartos dela?'
        }
    ]
)

In [8]:
run = client.beta.threads.runs.create_and_poll(
    thread_id=thread.id, assistant_id=assistant.id
)

messages = list(client.beta.threads.messages.list(thread_id=thread.id, run_id=run.id))

In [9]:
run.to_dict()

{'id': 'run_6pvr1nho2iFxwedNJGbfTk0g',
 'assistant_id': 'asst_TXgxoREIAnrXfZKSxf8bkJoR',
 'cancelled_at': None,
 'completed_at': 1721159794,
 'created_at': 1721159786,
 'expires_at': None,
 'failed_at': None,
 'incomplete_details': None,
 'instructions': 'Você é um atendente experiente de um hotel.     Responda perguntas variadas sobre hotéis, acomodações, pontos turíscos e outras     informações relevantes aos possíveis hóspedes.    \n',
 'last_error': None,
 'max_completion_tokens': None,
 'max_prompt_tokens': None,
 'metadata': {},
 'model': 'gpt-3.5-turbo',
 'object': 'thread.run',
 'parallel_tool_calls': True,
 'required_action': None,
 'response_format': 'auto',
 'started_at': 1721159786,
 'status': 'completed',
 'thread_id': 'thread_Pvsrt5EZFYZcwSVC6HkGuvqF',
 'tool_choice': 'auto',
 'tools': [{'type': 'file_search'}],
 'truncation_strategy': {'type': 'auto', 'last_messages': None},
 'usage': {'completion_tokens': 557,
  'prompt_tokens': 4340,
  'total_tokens': 4897},
 'temperat

In [20]:
print(messages[0].content[0].text.value)

A pousada em questão é a Pousada Porto De Galinhas, localizada em Porto de Galinhas, Brasil. Algumas das características e comodidades oferecidas pela pousada incluem:

- Piscina ao ar livre aberta o ano todo.
- Quartos com varanda.
- Quartos com ar condicionado, TV de tela plana, frigobar, guarda-roupa, cofre e vista para o pátio interno.
- Banheiro privativo com chuveiro, incluindo produtos de higiene pessoal gratuitos.
- Buffet de café-da-manhã com frutas frescas, sucos, pães e bolos.
- Atividades na área aconselhadas pela recepção, tais como passeios de buggy, surf e mergulho.

A pousada oferece diferentes tipos de quartos, tais como:
1. **Quarto Duplo**:
   - Tamanho: 20 metros quadrados
   - Cama: 1 cama de casal
   - Comporta até duas pessoas
   - Inclui amenities de banho gratuitas, banheiro privativo, ar-condicionado, TV de tela plana, frigobar, guarda-roupa, cofre e vista para o pátio interno.

2. **Quarto Triplo**:
   - Tamanho: 20 metros quadrados
   - Camas: 1 cama de solt

---

# Testing RAGApplication Class from rag.py 

In [22]:
VECTOR_STORE_ID = 'vs_UVad5hh6Y8gCsQ0OTH57nCuw'
ASSISTANT_ID = 'asst_TXgxoREIAnrXfZKSxf8bkJoR'

In [23]:
from rag import RAGApplication

rag_application = RAGApplication(os.getenv('OPENAI_API_KEY'), ASSISTANT_ID, VECTOR_STORE_ID)

In [24]:
rag_application.add_user_message('Qual é o nome dessa pousada?')

Message(id='msg_IqvGJTYgMHzOgzGPWCDec5Xa', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='Qual é o nome dessa pousada?'), type='text')], created_at=1721161267, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_dbKdzxGMQR0dFDDxEj2KfGlD')

In [25]:
res_dict = rag_application.run_thread()

In [26]:
res_dict['messages']

['O nome da pousada é Pousada Porto De Galinhas. Para mais informações, você pode acessar o [link](https://www.booking.com/hotel/br/pousada-porto-de-galinhas.pt-br.html) fornecido.']

In [27]:
rag_application.add_user_message('Qual foi minha última pergunta mesmo?')

Message(id='msg_nCLa7EahGlSRKDRZFujs8y45', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='Qual foi minha última pergunta mesmo?'), type='text')], created_at=1721161333, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_dbKdzxGMQR0dFDDxEj2KfGlD')

In [28]:
res_dict2 = rag_application.run_thread()
res_dict2['messages']

['Sua última pergunta foi sobre o nome da pousada. Posso ajudar com mais alguma informação?']

In [29]:
res_dict2 = rag_application.run_thread()
res_dict2['messages']

['Posso ajudar com mais alguma informação?']

In [32]:
res_dict2['run'].to_dict()

{'id': 'run_lrOjVkJqdWQGpdcBbQISB9zP',
 'assistant_id': 'asst_TXgxoREIAnrXfZKSxf8bkJoR',
 'cancelled_at': None,
 'completed_at': 1721161433,
 'created_at': 1721161431,
 'expires_at': None,
 'failed_at': None,
 'incomplete_details': None,
 'instructions': 'Você é um atendente experiente de um hotel.     Responda perguntas variadas sobre hotéis, acomodações, pontos turíscos e outras     informações relevantes aos possíveis hóspedes.    \n',
 'last_error': None,
 'max_completion_tokens': None,
 'max_prompt_tokens': None,
 'metadata': {},
 'model': 'gpt-3.5-turbo',
 'object': 'thread.run',
 'parallel_tool_calls': True,
 'required_action': None,
 'response_format': 'auto',
 'started_at': 1721161431,
 'status': 'completed',
 'thread_id': 'thread_dbKdzxGMQR0dFDDxEj2KfGlD',
 'tool_choice': 'auto',
 'tools': [{'type': 'file_search'}],
 'truncation_strategy': {'type': 'auto', 'last_messages': None},
 'usage': {'completion_tokens': 13, 'prompt_tokens': 787, 'total_tokens': 800},
 'temperature': 1